In [9]:
import pandas as pd
import json

In [ ]:
df_meps_raw = pd.read_csv('data/mep_detail.csv')


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [17]:
df_meps_raw['hasMembership'] = df_meps_raw['hasMembership'].str.replace("'", '"').apply(json.loads)


In [36]:
from datetime import datetime, timedelta
import pandas as pd
from collections import defaultdict
from tqdm import tqdm


def create_membership_timeline(mep_id, memberships_data):
    # Get all unique dates where changes happen
    dates = set()
    for membership in memberships_data:
        member_during = membership.get('memberDuring', {})
        start_date = member_during.get('startDate')
        end_date = member_during.get('endDate')
        
        if start_date:
            dates.add(start_date)
        if end_date:
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
            dates.add(end_dt.strftime('%Y-%m-%d'))
    
    dates = sorted(dates)
    
    # Initialize results structure
    timeline = []
    
    # For each date, check active memberships
    for date in dates:
        date_data = defaultdict(int)
        current_date = datetime.strptime(date, '%Y-%m-%d')
        
        # Add the MEP ID
        date_data['ID'] = mep_id
        date_data['date'] = date
        
        # Check each membership
        for membership in memberships_data:
            member_during = membership.get('memberDuring', {})
            start_date = member_during.get('startDate')
            end_date = member_during.get('endDate')
            
            # Check if membership is active on this date
            is_active = False
            if start_date:
                start_dt = datetime.strptime(start_date, '%Y-%m-%d')
                if end_date:
                    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
                    is_active = start_dt <= current_date <= end_dt
                else:
                    is_active = start_dt <= current_date
            
            if is_active:
                # Handle country representation
                if 'represents' in membership:
                    date_data['COUNTRY'] = membership['represents'].split('/')[-1]
                
                # Handle organization (like political group)
                org = membership.get('organization', '')
                if org:
                    org_type = membership.get('membershipClassification', '').split('/')[-1]
                    if org_type == 'EU_POLITICAL_GROUP':
                        date_data['POLITICAL_GROUP'] = org.split('/')[-1]
                
                # Handle role
                entity = membership.get('membershipClassification', '').split('/')[-1]
                role = membership.get('role', '').split('/')[-1]
                date_data[f"{entity} - {role}"] = 1
                
        
        timeline.append(dict(date_data))
    
    # Convert to DataFrame
    df = pd.DataFrame(timeline)
    
    return df

df_final = pd.DataFrame()
for index, row in tqdm(df_meps_raw.iterrows(), total=len(df_meps_raw)):
    mep_id = row['id'].split('/')[-1]
    df = create_membership_timeline(mep_id, row['hasMembership'])
    df_final = pd.concat([df_final, df])


100%|██████████| 4096/4096 [01:58<00:00, 34.68it/s]


In [47]:
df_final.sort_values(by=['ID', 'date'], inplace=True)

df_final.to_csv('./data/silver/mep_membership_timeline.csv', index=False)


In [51]:
df_final['POLITICAL_GROUP'].nunique()

88